In [129]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import spearmanr
from scipy.stats import ttest_ind
import seaborn as sns

In [130]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

1) Объедините два массива в DataFrame. 

In [131]:
petersburg_df = pd.DataFrame(petersburg, columns=['petersburg'])
#display(petersburg_df)
magadan_df = pd.DataFrame(magadan, columns=['magadan'])
#display(magadan_df)

df = petersburg_df.join(magadan_df, how='right')

df = df.fillna(value={'petersburg': df['petersburg'].median()}) # заполнение NaN медианным значением
display(df)

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


Проверка данных на нормальность

In [132]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально'

alpha = 0.05

_, p = shapiro(df) # Тест Шапиро-Уилка
print('p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.101
Данные распределены нормально


In [133]:
_, p = normaltest(df) # Тест Д’Агостино

print('p=%.3f' % p[0])

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.035
Данные распределены нормально


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Проверим каждый признак данных на нормальность

In [134]:
H0 = 'Признак распределен нормально'
Ha = 'Признак не распределен нормально'

_, p = shapiro(df['petersburg']) # Тест Шапиро-Уилка
print('p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

_, p = shapiro(df['magadan']) # Тест Шапиро-Уилка
print('p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.106
Признак распределен нормально
p=0.036
Признак не распределен нормально


In [135]:
#sns.histplot(df['petersburg'])

In [136]:
#sns.histplot(df['magadan'])

2) Обоснуйте выбор теста на корреляцию. 

Данные в совокупности распределены нормально, однако если рассматривать признаки в отдельности, то признак "magadan" не распределен нормально в отличие от признака "petersburg", распределение которого является близким к гауссовскому. Также переменные являются числовыми. Значит, учитывая эти факты, необходимо использовать корреляцию Спирмена для проведения теста на корреляцию.

In [137]:
corr, p = spearmanr(df['petersburg'], df['magadan'])
print(corr,p)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутствии зависимости между переменными.")

0.15569141404872366 0.7127617079991629
0.7127617079991629 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.


3) Сделайте вывод по гипотезе.

Исходя из корреляционного анализа можно сделать следующий вывод по гипотезе. Так, p-value больше уровня значимости - 0, 71 > 0,05, - это значит, что нельзя отвергуть нулевую гипотезу H0, состояющую в том, что зависимость между переменными отсутствует. Иначе говоря, при заданном уровне значимости альфа 5% можно говорить об отсутствии зависимости между двумя признаками (переменными).

4) Ответьте на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя.

In [138]:
H0 = 'Нет значимой разницы в среднем размере мидии в зависимости от города-производителя.'
Ha = 'Есть значимая разница в среднем размере мидии в зависимости от города-производителя.'

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)

    p = round(test_results[1],3)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.003 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница в среднем размере мидии в зависимости от города-производителя.


5) Обоснуйте выбор теста. 

Согласно результатам тестов на нормальность, а именно, тестов Шапиро-Уилка и Д’Агостино, данные распределены по нормальному закону. Значит рассматриваем параметрические тесты. Зависимая переменная (средний размер мидии) является количественной(числовой). Группы происходят из разных совокупностей (разные города - Санкт-Петербург и Магадан). Объектов в выборке меньше 30. Следовательно, мы используем независимый T-тест.

6) Сделайте вывод по гипотезе.

На основании независимого t-теста можно сделать вывод о том, что с заданным уровнем значимости альфа 5% нулевая гипоза отвергается и принимается альтернативная гипотеза, состоящая в том, что есть значимая статистическая разница в среднем размере мидии в зависимости от города-производителя. Иначе говоря, средние размеры мидий в городе Санкт-Петербурге и Магадане значимо отличаются друг от друга.